# CVA Calculation for Risky Bond
This exercise is taken from [HERE](https://www.finrgb.com/swatches/cva-calculation-for-risky-bond-solved-example/)
Credit: Bhuvnesh Khurana

Question:
A 3-year corporate bond pays a coupon of 8% per annum (once a year). The term structure of risk-free interest rates is flat at 4% p.a. and is expected to stay unchanged in the future. If the risk-neutral conditional probability of default of the issuer is 2% for a 1-year period and the recovery rate in event of default is 40% (assumed constant), the credit spread of this bond is CLOSEST to:
A) 120 bps
B) 125 bps
C) 130 bps
D) 135 bps

In [22]:
import numpy as np, numpy_financial as npf

## Expected exposure each year:
EE1 = 8 + 8/1.04 + 8/(1.04)^2 = 115.5444
EE2 = 8 + 8/1.04 = 111.8462
EE3 = 8 + 100

In [ ]:
EE = [115.5444, 111.8462, 108]

## Probability of Default each year:
PD1 = 2% = 0.02
PD2 = (1 - 0.02)(0.02) = 0.0196
PD3 = (1 - 0.02)^2 * (0.02) = 0.0192

The conditional default in each time period is 2%.
The survival probability is (1 - 0.02)

In [ ]:
PD = [0.02, 0.0196, 0.0192]

## Discount Factor in each time period:
DF1 = 1/1.04
DF1 = (1/1.04)^2
DF1 = (1/1.04)^3

In [ ]:
DF = [0.96153, 0.9246, 0.8890]

## CVA Equation
CVA = LGD * SUM(EE * PD * DF)

In [18]:
# The recovery rate is given to us as 40%. Therefore:
LGD = 0.6

# Basic CVA Equation
CVA = LGD*sum(x*y*z for x, y, z in zip(EE,PD,DF))
CVA

3.6553879622592

The equation for the Risky Value is:
Risky = Risk-Free - CVA

Now that we have the CVA, we use the Present Value formula for a risk-free bond to get the Risk-Free value.

In [31]:
# Calculate the Risk-Free Rate
# Present value of a risk-free bond
principal = 100
annual_interest = 0.04
period = 3
coupon_payment = 0.08 * principal

risk_free = (npf.pv(annual_interest, period, coupon_payment, principal)) * -1
risk_free

111.10036413290852

In [32]:
# The risky_value is:
risky_value = risk_free - CVA
risky_value

107.44497617064933

## Use the Risky Value to Find Yield to Maturity

Using a Yield to Maturity calculator found here:
https://dqydj.com/bond-yield-to-maturity-calculator/
**YTM = 5.253**

## Find the Credit-Spread
Since the risk-free interest rate is 4%. 
Credit Spread = Risky YTM - Risk-Free Int Rate

Credit Spread = 5.253 - 4 = 1.253%

**The answer is B**

## Notes on this Simple CVA Example

* The sum() and zip() function were useful when creating the sum of the product of multiple arrays.
* The npf.pv() function was very useful to simply finding the present value of a risk-free, coupon bond.
* The Credit Spread is the Risky YTM - Risk-Free Int Rate.

# EXTRA
## Class function for coupon bond pricing found [HERE](https://medium.com/@gennadii.turutin/how-to-calculate-yield-to-maturity-with-python-65a9a34d56f3)
Credit: Gennadi Turutin
-Although the formula does not seem to work, it is a good starting point.